Implementation of Aho-Corasick Algorithm

In [106]:
from collections import defaultdict


class AhoCorasick:
	def __init__(self, words):

		self.max_states = sum([len(word) for word in words])  #sum of the length of all words.
		self.max_characters = 26 #a-z
		self.output = [0]*(self.max_states+1) #output function
		self.fail_link = [-1]*(self.max_states+1) #failure link
		self.trie = [[-1]*self.max_characters for _ in range(self.max_states+1)]

		self.words = words
		self.states_count = self.match_state()


	def match_state(self):
		k = len(self.words)

		states = 1

		for i in range(k):
			word = self.words[i]
			current_state = 0

			for character in word:
				ch = ord(character) - 97 # Ascii value of a = 97

				if self.trie[current_state][ch] == -1:
					self.trie[current_state][ch] = states
					states += 1

				current_state = self.trie[current_state][ch]

			self.output[current_state] |= (1<<i)
   
		for ch in range(self.max_characters):
			if self.trie[0][ch] == -1:
				self.trie[0][ch] = 0
	
		queue = []

		for ch in range(self.max_characters):
			if self.trie[0][ch] != 0:
				self.fail_link[self.trie[0][ch]] = 0
				queue.append(self.trie[0][ch])

		while queue:
			state = queue.pop(0)

			for ch in range(self.max_characters):

				if self.trie[state][ch] != -1:

					failure = self.fail_link[state]
					while self.trie[failure][ch] == -1:
						failure = self.fail_link[failure]
					
					failure = self.trie[failure][ch]
					self.fail_link[self.trie[state][ch]] = failure

					self.output[self.trie[state][ch]] |= self.output[failure]

					queue.append(self.trie[state][ch])
		
		return states

	def find_nextstate(self, current_state, next_input):
		answer = current_state
		ch = ord(next_input) - 97 
		while self.trie[answer][ch] == -1:
			answer = self.fail_link[answer]

		return self.trie[answer][ch]

	def search(self, text):
	
		current_state = 0
		result = defaultdict(list)
		for i in range(len(text)):
			current_state = self.find_nextstate(current_state, text[i])

			if self.output[current_state] == 0: continue

			for j in range(len(self.words)):
				if (self.output[current_state] & (1<<j)) > 0:
					word = self.words[j]

					result[word].append(i-len(word)+1)
     
		return result

Test it here:

In [107]:
if __name__ == "__main__":
	words = ["es", "eskies", "ski"]
	text = "blueskies"

	aho_chorasick = AhoCorasick(words)
	result = aho_chorasick.search(text)
 	
	for word in result:
		for i in result[word]:
			print("The Keyword", word, "appears between", i, "to", i+len(word)-1)

The Keyword es appears between 3 to 4
The Keyword es appears between 7 to 8
The Keyword ski appears between 4 to 6
The Keyword eskies appears between 3 to 8


In [108]:
if __name__ == "__main__":
	words = ["potato", "tattoo", "theater","other"]
	text = "xxpotattooxx"

	aho_chorasick = AhoCorasick(words)
	result = aho_chorasick.search(text)
 	
	for word in result:
		for i in result[word]:
			print("The Keyword", word, "appears between", i, "to", i+len(word)-1)

The Keyword tattoo appears between 4 to 9
